In [ ]:
from PIL import Image

import numpy as np

from skimage.restoration import inpaint

def load_image(image_path):

    img = Image.open(image_path)

    return img

def mark_watermark_area(img, x1, y1, x2, y2):

    img_array = np.array(img)

    mask = np.zeros(img_array.shape[:2], dtype=np.uint8)

    mask[y1:y2, x1:x2] = 1

    return img_array, mask

def remove_watermark(img_array, mask):

    result = inpaint.inpaint_biharmonic(img_array, mask, multichannel=True)

    return result

def save_image(result, output_path):

    img_result = Image.fromarray(result.astype('uint8'))

    img_result.save(output_path)



In [ ]:
# 主程序

image_path = ''
output_path = ''
x1, y1, x2, y2 = 100, 100, 200, 200
img = load_image(image_path)
img_array, mask = mark_watermark_area(img, x1, y1, x2, y2)
result = remove_watermark(img_array, mask)
save_image(result, output_path)

In [ ]:
##代码test：生成斜纹半透明水印
from PIL import Image, ImageDraw, ImageFont
import math

# ========= 参数区 =========
input_image_path = "input.jpg"        # 原图路径
output_image_path = "watermarked.jpg" # 输出路径
watermark_text = "SAMPLE WATERMARK"   # 水印文字
watermark_opacity = 0.15              # 水印不透明度 alpha，0~1，建议 0.1~0.3
font_size = 40                        # 字体大小
angle_deg = 30                        # 斜纹角度（度数）
tile_spacing = 200                    # 水印之间的间距（像素）
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"  # 换成你本机的字体路径

# ========= 1. 打开原图，准备背景 B(x,y) =========
base = Image.open(input_image_path).convert("RGBA")  # RGBA 方便做 alpha 混合
W, H = base.size

# ========= 2. 创建一张“纯水印图层” =========
# 这张图相当于 W(x,y)，但先只是单色文字（白色），不含透明度
watermark_layer = Image.new("RGBA", (W, H), (0, 0, 0, 0))
draw = ImageDraw.Draw(watermark_layer)

try:
    font = ImageFont.truetype(font_path, font_size)
except IOError:
    # 如果字体路径不对，退回默认字体
    font = ImageFont.load_default()

# ========= 3. 在一个更大的画布上排布重复水印，再旋转 =========
# 为了旋转后不截掉边缘，这里做一个对角线长度的正方形画布
diag = int(math.sqrt(W*W + H*H))  # 对角线长度
tile_layer = Image.new("RGBA", (diag, diag), (0, 0, 0, 0))
tile_draw = ImageDraw.Draw(tile_layer)

# 在 tile_layer 上做网格排布的水印文字（还不旋转）
for y in range(0, diag, tile_spacing):
    for x in range(0, diag, tile_spacing):
        tile_draw.text((x, y), watermark_text, font=font, fill=(255, 255, 255, 255))

# 把这一大块水印图旋转一个角度，用 expand=True 保证完整
rotated = tile_layer.rotate(angle_deg, expand=True)

# ========= 4. 把旋转后的水印图裁剪回原图大小 =========
# 我们从旋转图的中心往外裁一个 W x H 的区域
rw, rh = rotated.size
left = (rw - W) // 2
top = (rh - H) // 2
cropped_watermark = rotated.crop((left, top, left + W, top + H))

# ========= 5. 控制整体透明度（构造 alpha(x,y)） =========
# 这里我们简单地对所有非透明像素统一乘以 watermark_opacity
# 这一步相当于给 W(x,y) 乘上一个全局 alpha
alpha_scaled = cropped_watermark.split()[3].point(
    lambda a: int(a * watermark_opacity)
)
watermark_with_alpha = cropped_watermark.copy()
watermark_with_alpha.putalpha(alpha_scaled)

# ========= 6. 把水印层叠加到原图上，得到 I_obs(x,y) =========
# PIL 的 alpha_composite 做的就是：
# I_obs = alpha * W + (1 - alpha) * B
composited = Image.alpha_composite(base, watermark_with_alpha)

# 如果你想保存成 JPG，可以去掉 alpha 通道
composited_rgb = composited.convert("RGB")
composited_rgb.save(output_image_path, quality=95)

print("Done! Saved to:", output_image_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, img_as_float
from skimage.restoration import inpaint

# ============ 1. 读取图片 ============
# 换成你自己的图片路径，例如 "example.jpg"
image_path = "example.jpg"

# skimage.io.imread 默认是 H x W x C
image = img_as_float(io.imread(image_path))  # 转成 float [0,1]
print("Image shape:", image.shape)

# ============ 2. 构造遮挡区域（mask） ============
# mask 为 True 的地方表示“需要修复的区域”
mask = np.zeros(image.shape[:2], dtype=bool)

H, W = mask.shape

# 这里我们简单地在中间画一个矩形遮挡
h_frac = 0.4   # 遮挡高度占比
w_frac = 0.4   # 遮挡宽度占比

h_box = int(H * h_frac)
w_box = int(W * w_frac)

top = H // 2 - h_box // 2
left = W // 2 - w_box // 2

mask[top:top + h_box, left:left + w_box] = True

# 为了直观展示“被遮挡”的效果，我们把这块画成纯色（比如黑色）
occluded_image = image.copy()
occluded_image[mask] = 0.0   # 全黑；也可以改成别的颜色

# ============ 3. 调用 inpaint_biharmonic 做图像修复 ============
# 注意：新版 skimage 用 channel_axis，而不是 multichannel
# channel_axis=-1 表示颜色通道在最后一个维度 (H, W, C)
inpainted_image = inpaint.inpaint_biharmonic(
    occluded_image,
    mask,
    channel_axis=-1
)

# ============ 4. 可视化结果 ============
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
ax = axes.ravel()

ax[0].imshow(image)
ax[0].set_title("Original")
ax[0].axis("off")

ax[1].imshow(occluded_image)
ax[1].set_title("With Occlusion")
ax[1].axis("off")

ax[2].imshow(inpainted_image)
ax[2].set_title("Inpainted")
ax[2].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_float

def fft_spectrum(channel):
    """
    对单个灰度通道做 2D FFT，并返回：
    - 原始 FFT f
    - 经过 fftshift 移到中心的频谱 fshift
    - 对数幅度谱 mag (便于显示)
    """
    f = np.fft.fft2(channel)
    fshift = np.fft.fftshift(f)
    mag = np.log(np.abs(fshift) + 1e-8)
    return f, fshift, mag

# ===== 1. 读取 PNG 彩色图像 =====
img = io.imread("input.png")   # 换成你的 PNG 路径
img = img_as_float(img)

print("Original image shape:", img.shape)

# 如果有 alpha 通道（RGBA），丢掉 alpha，只保留 RGB
if img.ndim == 3 and img.shape[2] == 4:
    img_rgb = img[:, :, :3]
elif img.ndim == 3 and img.shape[2] == 3:
    img_rgb = img
else:
    raise ValueError("图像不是标准的 RGB/RGBA 彩色图，当前 shape = {}".format(img.shape))

R = img_rgb[:, :, 0]
G = img_rgb[:, :, 1]
B = img_rgb[:, :, 2]

# ===== 2. 对 R/G/B 三个通道分别做 FFT =====
f_R, fshift_R, mag_R = fft_spectrum(R)
f_G, fshift_G, mag_G = fft_spectrum(G)
f_B, fshift_B, mag_B = fft_spectrum(B)

# ===== 3. 做一些简单的一致性“度量” =====
# 例如：三通道幅度谱的平均值，以及两两差分
mag_mean = (mag_R + mag_G + mag_B) / 3.0
diff_RG = np.abs(mag_R - mag_G)
diff_RB = np.abs(mag_R - mag_B)
diff_GB = np.abs(mag_G - mag_B)

# 也可以粗暴一点，看“三通道都很像”的地方：
# 比如：max diff < 某个小阈值，标记为“高一致性区域”------------------------------------------------!
max_diff = np.maximum.reduce([diff_RG, diff_RB, diff_GB])
consistency_mask = max_diff < 0.05  # 阈值可调，这里只是示意用

# ===== 4. 可视化：原图 + R/G/B 通道 + 频谱 + 一致性掩膜 =====
fig, axes = plt.subplots(3, 4, figsize=(14, 10))
ax = axes.ravel()

# 原图
ax[0].imshow(img_rgb)
ax[0].set_title("Original RGB")
ax[0].axis("off")

# R/G/B 通道（空间域）
ax[1].imshow(R, cmap="gray")
ax[1].set_title("R channel (spatial)")
ax[1].axis("off")

ax[2].imshow(G, cmap="gray")
ax[2].set_title("G channel (spatial)")
ax[2].axis("off")

ax[3].imshow(B, cmap="gray")
ax[3].set_title("B channel (spatial)")
ax[3].axis("off")

# R/G/B 频谱（对数幅度）
ax[4].imshow(mag_R, cmap="gray")
ax[4].set_title("R channel spectrum")
ax[4].axis("off")

ax[5].imshow(mag_G, cmap="gray")
ax[5].set_title("G channel spectrum")
ax[5].axis("off")

ax[6].imshow(mag_B, cmap="gray")
ax[6].set_title("B channel spectrum")
ax[6].axis("off")

# 三通道平均频谱
ax[7].imshow(mag_mean, cmap="gray")
ax[7].set_title("Mean spectrum (R+G+B)/3")
ax[7].axis("off")

# 两两差分（频谱差异）
ax[8].imshow(diff_RG, cmap="magma")
ax[8].set_title("|R - G| spectrum diff")
ax[8].axis("off")

ax[9].imshow(diff_RB, cmap="magma")
ax[9].set_title("|R - B| spectrum diff")
ax[9].axis("off")

ax[10].imshow(diff_GB, cmap="magma")
ax[10].set_title("|G - B| spectrum diff")
ax[10].axis("off")

# “一致性掩膜”可视化（这里只是示意）
ax[11].imshow(consistency_mask, cmap="gray")
ax[11].set_title("High-consistency mask\n(3 channels similar)")
ax[11].axis("off")

plt.tight_layout()
plt.show()


In [ ]:

def reconstruct_rgb_from_spectra(fshift_R, fshift_G, fshift_B, normalize=True):
    # R 通道
    f_R = np.fft.ifftshift(fshift_R)
    img_R_complex = np.fft.ifft2(f_R)
    img_R = np.real(img_R_complex)

    # G 通道
    f_G = np.fft.ifftshift(fshift_G)
    img_G_complex = np.fft.ifft2(f_G)
    img_G = np.real(img_G_complex)

    # B 通道
    f_B = np.fft.ifftshift(fshift_B)
    img_B_complex = np.fft.ifft2(f_B)
    img_B = np.real(img_B_complex)

    img_rgb = np.stack([img_R, img_G, img_B], axis=-1)

    if normalize:
        img_min = img_rgb.min()
        img_max = img_rgb.max()
        if img_max > img_min:
            img_rgb = (img_rgb - img_min) / (img_max - img_min)

    return img_rgb


In [ ]:
mask = consistency_mask
fs_R = fshift_R * mask
fs_G = fshift_G * mask
fs_B = fshift_B * mask

img = reconstruct_rgb_from_spectra(fs_R, fs_G, fs_B, normalize=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 10))
ax = axes.ravel()

# 原图
ax[0].imshow(img_rgb)
ax[0].set_title("Original RGB")
ax[0].axis("off")

# R/G/B 通道（filtered）
ax[1].imshow(img)
ax[1].set_title("Filtered")
ax[1].axis("off")

plt.tight_layout()
plt.show()

In [55]:

import sys
sys.path.append("") 

In [58]:
import torch
from torchvision import transforms
from PIL import Image
from watermark_remover import WatermarkRemover
import numpy as np

image_path = "input256.png"  # Replace with the path to your test image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = WatermarkRemover().to(device)
model_path = "/Users/zouxd/Documents/--Tools--/-img/Watermark_Removal/model.pth"  # Replace with the path to your saved model
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.ToTensor(),])
watermarked_image = Image.open(image_path).convert("RGB")
original_size = watermarked_image.size  
input_tensor = transform(watermarked_image).unsqueeze(0).to(device)

with torch.no_grad():
    output_tensor = model(input_tensor)

predicted_image = output_tensor.squeeze(0).cpu().permute(1, 2, 0).clamp(0, 1).numpy()
predicted_pil = Image.fromarray((predicted_image * 255).astype(np.uint8))
predicted_pil = predicted_pil.resize(original_size, Image.Resampling.LANCZOS)
predicted_pil.save("/Users/zouxd/Documents/--Tools--/predicted_image.jpg", quality=100)

